In [1]:
import os
#if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#    raise 'stop'
if not os.path.isdir('d:/flagellar/'):
    deps_path = '/kaggle/usr/lib/flg-packages/'
    !pip install --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt
    !tar xfvz /kaggle/input/ultralytics-for-offline-install/archive.tar.gz
    !pip install --no-index --find-links=./packages ultralytics
    !rm -rf ./packages

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import flg_yolo
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy

fast_mode = False
if fls.is_submission:
    fast_mode = False

MainProcess


In [3]:
all_data = fls.load_all_extra_data()

In [4]:
# np.random.default_rng(seed=0).shuffle(all_data)

# # Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
# N=100
# n_motors = np.array([len(d.labels) for d in all_data])
# inds_zero = np.argwhere(n_motors==0)[:N,0]
# inds_one = np.argwhere(n_motors==1)[:N,0]
# inds_test = np.concatenate((inds_zero,inds_one))
# inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
# inds_test.shape, inds_train.shape

# train_data = []
# for i in inds_train:
#     train_data.append(all_data[i])
# test_data = []
# for i in inds_test:
#     test_data.append(all_data[i])
# np.random.default_rng(seed=0).shuffle(test_data)
# test_data = test_data
# if fast_mode:
#     train_data = train_data[1:3]
#     test_data = test_data[1:3]
# len(train_data), len(test_data)
train_data = all_data
test_data = all_data

In [5]:
fls.profiling=False
model = flg_yolo.YOLOModel()
model.run_in_parallel = True
model.train(train_data, test_data)
fls.dill_save(fls.temp_dir+'yolo_temp.pickle', model)

CompletedProcess(args=['pip', 'uninstall', '-y', 'albumentations'], returncode=0)
Will process approximately 765 slices for training


Processing training motors:   0%|          | 0/85 [00:00<?, ?it/s]

Clearing pytorch
Will process approximately 765 slices for validation


Processing validation motors:   0%|          | 0/85 [00:00<?, ?it/s]


Preprocessing Complete:
- Training data: 83 tomograms, 85 motors, 765 slices
- Validation data: 83 tomograms, 85 motors, 765 slices
- Dataset directory: d:/flagellar/temp//yolo_dataset/
- YAML configuration: d:/flagellar/temp//yolo_dataset/dataset.yaml

Ready for YOLO training!
Starting YOLO training process...
Created new YAML at d:/flagellar/temp/training.yaml
Using YAML file: d:/flagellar/temp/training.yaml
YAML contents:
names:
  0: motor
path: d:/flagellar/temp//yolo_dataset/
train: images/train
val: images/val


Starting YOLO training...
New https://pypi.org/project/ultralytics/8.3.102 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.98  Python-3.10.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=d:/flagellar/temp/training.yaml, epochs=30, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=4, project=d:/flagellar/temp//yolo_weights

...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:06

                   all        765        765      0.955      0.841      0.931      0.524



30 epochs completed in 0.150 hours.
Optimizer stripped from d:\flagellar\temp\yolo_weights\motor_detector\weights\last.pt, 52.1MB
Optimizer stripped from d:\flagellar\temp\yolo_weights\motor_detector\weights\best.pt, 52.1MB

Validating d:\flagellar\temp\yolo_weights\motor_detector\weights\best.pt...
Ultralytics 8.3.98  Python-3.10.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs



                Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:09

                   all        765        765      0.863      0.868      0.915      0.598
Speed: 0.2ms preprocess, 7.7ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to d:\flagellar\temp\yolo_weights\motor_detector
Loss curve saved to d:/flagellar/temp//yolo_weights/motor_detector\dfl_loss_curve.png

Best model found at epoch 736 with validation DFL loss: 0.0000

Training complete!


In [9]:
tt = []
for t in test_data:
    if len(t.labels)==1:
        tt.append(t)
inferred_test_data = model.infer(tt)
print(fls.score_competition_metric(inferred_test_data, tt))

0.9801488833746899


In [10]:
all_data = fls.load_all_train_data()
np.random.default_rng(seed=0).shuffle(all_data)

# Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
N=20
n_motors = np.array([len(d.labels) for d in all_data])
inds_zero = np.argwhere(n_motors==0)[:N,0]
inds_one = np.argwhere(n_motors==1)[:N,0]
inds_test = np.concatenate((inds_zero,inds_one))
inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
inds_test.shape, inds_train.shape

train_data = []
for i in inds_train:
    train_data.append(all_data[i])
test_data = []
for i in inds_test:
    test_data.append(all_data[i])
np.random.default_rng(seed=0).shuffle(test_data)
test_data = test_data
if fast_mode:
    train_data = train_data[1:3]
    test_data = test_data[1:3]
len(train_data), len(test_data)

(604, 40)

In [11]:
inferred_test_data = model.infer(test_data)
print(fls.score_competition_metric(inferred_test_data, test_data))

0.6451612903225806


In [7]:
model.run_in_parallel = True